In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import requests
import json
from config import gmap_api_key
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
mental = pd.read_csv('Resources/mental.csv')
vaccine = pd.read_csv('Resources/vaccine_trimmed.csv', low_memory=False)

# Data cleaning for Gender and Sexual orientation

In [5]:
orientation = mental.loc[mental["Group"]=="By Sexual orientation",:]
orientation["Time Period Start Date"]=pd.to_datetime(orientation["Time Period Start Date"])
orientation["Time Period End Date"]=pd.to_datetime(orientation["Time Period End Date"])
orientation = orientation.sort_values("Time Period Start Date").reset_index().drop(columns="Quartile Range")
depression_orientation = orientation.loc[orientation["Indicator"]=="Symptoms of Depressive Disorder",:]
anxiety_orientation=orientation.loc[orientation["Indicator"]=="Symptoms of Anxiety Disorder",:]
depression_orientation = depression_orientation.dropna()
anxiety_orientation = anxiety_orientation.dropna()
depression_orientation.to_csv("Resources/depression_orientation.csv")
anxiety_orientation.to_csv("Resources/anxiety_orientation.csv")
gender = mental.loc[mental["Group"]=="By Sex",:]
gender["Time Period Start Date"]=pd.to_datetime(gender["Time Period Start Date"])
gender["Time Period End Date"]=pd.to_datetime(gender["Time Period End Date"])
gender = gender.sort_values("Time Period Start Date").reset_index().drop(columns="Quartile Range")
depression_gender = gender.loc[gender["Indicator"]=="Symptoms of Depressive Disorder",:]
anxiety_gender=gender.loc[gender["Indicator"]=="Symptoms of Anxiety Disorder",:]
depression_gender = depression_gender.dropna()
anxiety_gender = anxiety_gender.dropna()
depression_gender.to_csv("Resources/depression_gender.csv")
anxiety_gender.to_csv("Resources/anxiety_gender.csv")

# Vaccination Rates in States vs Mental Wellbeing Cleaning

In [6]:
#read in csv data from datasets taken from:
#1.) Mental is based off of the CDC mental health survey found here https://data.cdc.gov/NCHS/Indicators-of-Anxiety-or-Depression-Based-on-Repor/8pt5-q6wp
#2.) Vaccination Data is taken from the CDC found here  https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh
mental = pd.read_csv('Resources/mental.csv')
vaccine = pd.read_csv('Resources/vaccine_trimmed.csv')
vaccine = vaccine.drop(columns =['Unnamed: 0'])

In [7]:
#Filtering the data based off of the indicator which determines which symptoms are counted as a positive result from the questionaire. 
#I have chosen both anxiety and depression as my subject of interest as I more focused on overall mental unrest
mental_trimmed = mental.loc[(mental['Group'] == 'By Age') | (mental['Group'] == 'By State')].reset_index(drop=True)
mental_trimmed = mental_trimmed.loc[mental_trimmed['Indicator'] == "Symptoms of Anxiety Disorder or Depressive Disorder"]

In [8]:
#After careful examination of the datasets involved in this analysis. The vaccine dataframe conveys data from all days from
#12/13/2020 to 6/13/2022. However the mental health data is taken from specific time frames and then reported as a collective.
#Due to these differences and the inability to know exactly when the mental health data was taken the vaccination data was used
#for solely the last day of a given time period in the mental health survey. This code is finding those dates and trimming them
#down until there is both mental health and vaccination status data from both dataframes.
end_dates = mental['Time Period End Date'].unique().tolist()
end_trim = end_dates[0:-1]
end_values = end_trim[21:]
end_values.remove('01/05/2021')
end_values.remove('04/13/2021')
end_values.remove('07/20/2021')
end_values.remove('11/30/2021')
end_values.remove('03/01/2022')
end_values.remove('05/31/2022')

In [10]:
#This code is used to match state names to state codes. Since each dataframe is going to compared along a date from a given 
#state it is important that we have each dataframe arranged the same.
state_2 = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

state_2 = {state: abbrev for state, abbrev in state_2.items()}
new_mental = mental_trimmed.loc[mental_trimmed['Group'] == 'By State']
new_mental = new_mental.rename(columns={'Time Period End Date': 'Date'})
new_mental = new_mental[['State', 'Value', 'Date']]
new_mental['abbrev'] = new_mental['State'].map(state_2)
new_mental = new_mental.drop(columns=['State'])
new_mental = new_mental.rename(columns={'abbrev':"State"})


In [12]:
#In this cell we are working to have the dataframe come more in line with each other. This trims the columns further until
#only the interesting parts will remain. In addition to this, having a common framework will enable easier integration.
vax_clean = vaccine
vax_clean = vax_clean.rename(columns={'Recip_State': 'State'})
new_vax = pd.DataFrame()
mental_dates = pd.DataFrame()
for column in vax_clean.columns:
    new_vax[column] = ''
for column in new_mental.columns:
    mental_dates[column] = ''
for date in end_values:
    df = new_mental.loc[new_mental['Date'] == date]
    df_2 = vax_clean.loc[vax_clean['Date'] == date]
    mental_dates = pd.concat([mental_dates, df])
    new_vax = pd.concat([new_vax, df_2])

In [13]:
#While the columns are broadly speaking in alignment there is still far to much data in the vaccine information to be parsed
#easily. As such this cell removes the bulk of that unnecessary information.
trim_vax = pd.DataFrame()
state_list = vax_clean['State'].unique().tolist()
new_state_list = state_list[0:-2]
for column in new_vax.columns:
    trim_vax[column] = ''
for date in end_values:
    date_trim = new_vax.loc[new_vax['Date'] == date]
    for state in new_state_list:
        state_trim = date_trim.loc[date_trim['State'] == state]
        complete_trim = state_trim.iloc[2:3,:]
        #print(complete_trim)
        trim_vax = pd.concat([trim_vax, complete_trim])
trim_vax = trim_vax.dropna()
trim_vax = trim_vax.drop(columns=['Recip_County']).reset_index(drop=True)
trim_vax['Date'] = pd.to_datetime(trim_vax['Date'])
trim_vax['Date'] = trim_vax['Date'].dt.strftime("%Y/%m/%d")
trim_vax = trim_vax.sort_values(by=['Date','State'])
trim_vax = trim_vax.astype({'Date': str})
trim_vax = trim_vax.reset_index(drop=True)
trim_vax = trim_vax.loc[(trim_vax['State'] != 'PR') & (trim_vax['State'] != 'VI')]
trim_vax = trim_vax.reset_index(drop=True)

In [15]:
#Now that the vaccine information is trimmed to desired level(one date per state) the mental health dataframe is examined
mental_dates = mental_dates.reset_index(drop=True)

In [16]:
#Further information is determiend to not be overlapped primarily that DC is not accounted for the vaccination data.
#In addition there are three dates where HI does not report their vaccination data so these are removed for the mental health
#dataframe as well.
date_trim = mental_dates.loc[mental_dates['State'] != 'DC']
date_trim['Date'] = pd.to_datetime(mental_dates['Date'])
date_trim['Date'] = date_trim['Date'].dt.strftime("%Y/%m/%d")
date_trim = date_trim.sort_values(by=['Date','State'])
date_trim = date_trim.astype({'Date': str})
date_trim = date_trim.drop([1160,1210,1260])
date_trim = date_trim.reset_index(drop=True)

In [17]:
#Appending the matching data into the vaccination dataframe
value = date_trim['Value']
trim_vax['Value'] =value

In [23]:
#Write to CSV
trim_vax.to_csv('Resources/vaccine_data_df_2.csv', encoding='utf-8')

### Data extract and clean up for analysis of "Anxiety Disorder or Depressive Disorder"
##### by RG

In [18]:
# Save the csv data to a dataframe
mental = pd.read_csv('Resources/mental.csv')
print (f"len(mental): {len(mental)}")

# Extract and clean the data needed for analysis of "Anxiety Disorder or Depressive Disorder" 
indicator = "Symptoms of Anxiety Disorder or Depressive Disorder"

# Extract the columns needed for analysis
mental_data = mental.loc[  (mental["Indicator"] == indicator) 
                         , [ "Group"
                           , "Subgroup"
                           , "Value"
                           , "Time Period Start Date"
                           , "Time Period Label"]
                        ].reset_index(drop=True)

# Remove rows with null values
mental_data = mental_data.dropna()

# Convert sort the data by 'Time Period Start Date' 
mental_data['Date'] = pd.to_datetime(mental_data['Time Period Start Date'])
mental_data         = mental_data.sort_values(by=['Date'])
mental_data         = mental_data.astype({'Date':str})

#Export the mental_data into a csv file
output_path      = os.path.join("Resources/mental_data_Anxiety_and_Depressive_disorder.csv" )
mental_data.to_csv(output_path,index="False")
#mental_data.to_csv(output_path,index="True")
mental_data

len(mental): 10701


,Group,Subgroup,Value,Time Period Start Date,Time Period Label,Date
0,National Estimate,United States,35.9,04/23/2020,"Apr 23 - May 5, 2020",2020-04-23
50,By State,New Mexico,34.6,04/23/2020,"Apr 23 - May 5, 2020",2020-04-23
49,By State,New Jersey,37.6,04/23/2020,"Apr 23 - May 5, 2020",2020-04-23
48,By State,New Hampshire,34.8,04/23/2020,"Apr 23 - May 5, 2020",2020-04-23
47,By State,Nevada,34.8,04/23/2020,"Apr 23 - May 5, 2020",2020-04-23
...,...,...,...,...,...,...
3511,By Education,High school diploma or GED,35.3,06/29/2022,"Jun 29 - Jul 11, 2022",2022-06-29
3510,By Education,Less than a high school diploma,39.0,06/29/2022,"Jun 29 - Jul 11, 2022",2022-06-29
3509,By Race/Hispanic ethnicity,"Non-Hispanic, other races and multiple races",43.0,06/29/2022,"Jun 29 - Jul 11, 2022",2022-06-29
3516,By State,Alabama,37.4,06/29/2022,"Jun 29 - Jul 11, 2022",2022-06-29


In [22]:
byage = mental.loc[(mental["Indicator"]=="Symptoms of Anxiety Disorder or Depressive Disorder")&(mental["Group"]=="By Age"),:]
byage["Time Period Start Date"]=pd.to_datetime(byage["Time Period Start Date"])
byage["Time Period End Date"]=pd.to_datetime(byage["Time Period End Date"])
byage=byage.drop(columns="Quartile Range")
byage=byage.dropna()
byage.to_csv("Resources/byage.csv")
byeducation = mental.loc[(mental["Indicator"]=="Symptoms of Anxiety Disorder or Depressive Disorder")&(mental["Group"]=="By Education"),:]
byeducation["Time Period Start Date"]=pd.to_datetime(byeducation["Time Period Start Date"])
byeducation["Time Period End Date"]=pd.to_datetime(byeducation["Time Period End Date"])
byeducation=byeducation.drop(columns="Quartile Range")
byeducation=byeducation.dropna()
byeducation.to_csv("Resources/byeducation.csv")

# Data cleaning for State Analysis

In [19]:
anxiety_depression = mental.loc[(mental["Group"] == "By State") \
                                     &(mental["Indicator"] == \
                                       "Symptoms of Anxiety Disorder or Depressive Disorder"),:]
anxiety_depression_states = pd.DataFrame({"State":anxiety_depression["State"].unique(),"Lat":"","Lng":""})
for index,row in anxiety_depression_states.iterrows():
    state = row["State"]
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={state}&key={gmap_api_key}"
    response = requests.get(target_url).json()
    try:
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']
        anxiety_depression_states.loc[index,"Lat"]=lat
        anxiety_depression_states.loc[index,"Lng"]=lng
    except:
        print(f"{state} not found. Skipping...")
        pass
state_anxiety_depression = pd.merge(anxiety_depression,anxiety_depression_states, how="left",on="State")
state_anxiety_depression
state_anxiety_depression["Time Period Start Date"]=pd.to_datetime(state_anxiety_depression["Time Period Start Date"])
state_anxiety_depression["Time Period End Date"]=pd.to_datetime(state_anxiety_depression["Time Period End Date"])
vote_count = pd.read_csv("Resources/2020_vote.csv")
hpsa_df = pd.read_csv("Resources/HPSA.csv")
state_dataset = pd.merge(state_anxiety_depression,vote_count,on="State")
state_dataset=pd.merge(state_dataset,hpsa_df,on="State")
state_dataset.to_csv("Resources/state_dataset.csv",index=False)